# Mónadas Útiles

En general, tenemos que las mónadas nos permiten aplicar funciones a valores encapsulados, que a su vez encapsulan el resultado:

```haskell
(>>=) :: Monad m => m a -> (a -> m b) -> m b
```

En particular, ese contexto que encapsula a la mónada puede ser una tupla con un valor definido. Por lo tanto, podemos tener mónadas con función `>>=` de la forma:

```haskell
(>>=) :: (a, c) -> (a -> (b, c)) -> (b, c)
```

Y por lo tanto, podemos utilizar el primer valor de la tupla como el resultado de nuestras funciones, y el segundo para pasar información entre funciones. Esto se utiliza como base para todas las siguientes mónadas.

Para instalar varias instancias de Mónadas útiles:

In [ ]:
:! stack install mtl

## Reader

Nos sirve para compartir información entre llamadas de funciones.

Casos de uso:
- Valores de entorno.
- Configuración.
- Constantes.

Funciones:
- `ask`: Devuelve los valores almacenados.
- `asks`: Ejecuta una función que requiere los valores almacenados.

[Más información - Documentación](https://hackage.haskell.org/package/mtl-2.2.2/docs/Control-Monad-Reader.html)

In [ ]:
import Control.Monad.Reader

data ConfigBDD = ConfigBDD { database :: String
                           , username :: String
                           , table    :: String
                           , engine   :: String
                           } deriving Show

-- Returns True if the "count" variable contains correct bindings size.
conexion :: ConfigBDD -> String
conexion config = runReader generaURL config

-- The Reader monad, which implements this complicated check.
generaURL :: Reader ConfigBDD String
generaURL = do
    protocol <- asks generaProtocolo
    config <- ask
    return (protocol ++ username config ++ "@" ++ database config ++ "/" ++ table config)

-- Regresa un valor de la configuración.
generaProtocolo :: ConfigBDD -> String
generaProtocolo config = case engine config of
                            "postgresql" -> "postgresql://"
                            "mysql"      -> "mysql://"
                            "mongo"      -> "mongodb://"

-- Configuración de mi base de datos.
configuracion :: ConfigBDD
configuracion = ConfigBDD { database = "localhost:5432"
                          , username = "admin"
                          , table    = "usuarios"
                          , engine   = "postgresql"
                          }

main :: IO ()
main = do
    putStrLn "Información de BDD: "
    putStrLn $ show configuracion ++ "\n"
    putStr "URL de conexión: "
    putStrLn $ conexion configuracion
    
main

## Writer

Nos sirve para generar información a la par que aplicamos funciones.

Casos de uso:
- Logging
- Serialización
- Persistencia

Funciones:
- `writer`: Dada la tupla `(a, b)`, añade `b` a la información secundaria y regresa `a` dentro de una mónada `Writer`.
- `tell`: Añade su entrada a la información secundaria.

[Más información - Documentación](https://hackage.haskell.org/package/mtl-2.2.2/docs/Control-Monad-Writer-Lazy.html)

In [ ]:
import Control.Monad.Writer

mapLog :: (Show a, Show b) => (a -> b) -> [a] -> [b] -> Writer [String] [b]
mapLog _ []     ys = writer (reverse ys, ["Caso base"])
mapLog f (x:xs) ys = do
        tell ["Mapeando " ++ show x ++ " a " ++ show (f x)]
        mapLog f xs (f x:ys)

main :: IO ()
main = print $ runWriter (mapLog (+1) [1..5] [])

main

## State

Es una combinación entre `Reader` y `Writer` puesto que podemos tanto almacenar valores, como agregar más a futuro.

Casos de uso:
- Estados globales.

Funciones:
- `get`: Regresa el estado actual.
- `put`: Define el nuevo estado.

[Más información - Documentación](https://hackage.haskell.org/package/mtl-2.2.2/docs/Control-Monad-State-Lazy.html)

In [ ]:
import Control.Monad.State

stateGCD :: Integer -> Integer -> Integer
stateGCD x y = fst $ runState gcdRecursive (x, y)

gcdRecursive :: State (Integer, Integer) Integer
gcdRecursive = do
  (x, y) <- get
  case compare x y of
    EQ -> return x
    LT -> put (x, y - x) >> gcdRecursive
    GT -> put (x - y, y) >> gcdRecursive
    
stateGCD 5 7
stateGCD 4 6

In [ ]:
import Control.Monad.State

type Stack = [Int]

pop :: State Stack Int
pop = state $ \(x:xs) -> (x, xs)

push :: Int -> State Stack ()
push a = state $ \xs -> ((), a:xs)

modificaPila :: State Stack Int  
modificaPila = do  
    push 2
    push 1
    pop
    
runState modificaPila [3..5]

## IO

Mónada que nos enlaza con el "mundo real".

Casos de uso:
- Manejo de archivos.
- Imprimir en terminal.
- Manejar la entrada estándar.
- Utilizar internet u otros puertos.

Funciones:
- `putChar`: Imprime un caracter.
- `putStr`: Imprime una cadena de texto.
- `putStrLn`: Imprime una cadena de texto seguida de una nueva línea.
- `print`: Imprime algo que sea instancia de `Show`.
- `getChar`: Lee un caracter de la entrada estándar.
- `getLine`: Lee una línea de la entrada estándar.
- `readFile`: Lee un archivo dado su path.
- `writeFile`: Escribe una cadena de texto en un archivo dado su path.
- `appendFile`: Escribe al final de un archivo.

In [ ]:
main :: IO ()
main = do
    writeFile "test.txt" "Hola Mundo!"
    content <- readFile "test.txt"
    putStrLn content
    
main